In [4]:
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import datetime

In [5]:
data = pd.read_csv('dataset/lotto_his.csv')
data.head()

,first,second,third,fourth,fifth,sixth,bonus
0,3,7,10,13,25,36,32
1,16,21,28,35,39,43,12
2,10,34,38,40,42,43,32
3,11,17,19,21,22,25,24
4,4,8,18,25,27,32,42


In [34]:
fr_n = data['first'].values
se_n = data['second'].values
th_n = data['third'].values
fo_n = data['fourth'].values
fi_n = data['fifth'].values
si_n = data['sixth'].values
bo_n = data['bonus'].values
mean_v = (fr_n+se_n+th_n+fr_n+fi_n+si_n+bo_n)/7

In [23]:
seq_len=200
sequence_length = seq_len + 1

result=[]
for index in range(len(fi_n)-seq_len):
    result.append(fi_n[index:index + sequence_length])

In [24]:
normalized_data=[]
for window in result:
    normalized_window=[((float(p)/float(window[0]))-1)for p in window]
    normalized_data.append(normalized_window)

result = np.array(normalized_data)


row = int(round(result.shape[0]*0.9))
train = result[:row,:]
np.random.shuffle(train)

x_train = train[:,:-1]
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
y_train = train[:,-1]

x_test = result[row:,:-1]
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
y_test = result[row:,-1]

x_train.shape, x_test.shape

((598, 200, 1), (67, 200, 1))

In [26]:
model = Sequential()

model.add(LSTM(200,return_sequences=True, input_shape=(200,1)))
model.add(LSTM(128,return_sequences=False))
model.add(Dense(1,activation='linear'))
model.compile(loss='mse',optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200, 200)          161600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 330,177
Trainable params: 330,177
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200, 200)          161600    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dens

In [27]:
model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         batch_size=32,
         epochs=5)

Train on 598 samples, validate on 67 samples
Epoch 1/5
598/598 [==============================] - 53s 89ms/step - loss: 0.0701 - val_loss: 0.1040
Epoch 2/5
598/598 [==============================] - 52s 87ms/step - loss: 0.0629 - val_loss: 0.0992
Epoch 3/5
598/598 [==============================] - 52s 87ms/step - loss: 0.0635 - val_loss: 0.1105
Epoch 4/5
598/598 [==============================] - 52s 87ms/step - loss: 0.0626 - val_loss: 0.1069
Epoch 5/5
598/598 [==============================] - 52s 87ms/step - loss: 0.0610 - val_loss: 0.1012
Train on 598 samples, validate on 67 samples
Epoch 1/5
598/598 [==============================] - 54s 90ms/step - loss: 0.0427 - val_loss: 0.0830
Epoch 2/5
598/598 [==============================] - 52s 86ms/step - loss: 0.0220 - val_loss: 0.0782
Epoch 3/5
598/598 [==============================] - 52s 87ms/step - loss: 0.0208 - val_loss: 0.0769
Epoch 4/5
598/598 [==============================] - 52s 87ms/step - loss: 0.0234 - val_loss: 0.0672
E

In [33]:
pred = model.predict(x_test)

(y_test[65]+1)*window[0],(int)((pred[65,0]+1)*window[0]), window[200]


(18.18181818181818, 16, 0, 0, 16.128205128205128, 16)